In [34]:
# Jhose Immanuel Sembiring
# 1103202047
# TK44G4
# Modern-CNN_GoogleNet

#Library

In [35]:
import torch
from torch import nn
import torch.nn.functional as F


#Kelas Inception Block

In [36]:
# Inception block implementation
class Inception(nn.Module):
    def __init__(self, in_channels, c1, c2_in, c2_out, c3_in, c3_out, c4_out):
        super(Inception, self).__init__()
        # Branch 1
        self.b1 = nn.Conv2d(in_channels, c1, kernel_size=1)

        # Branch 2
        self.b2_1 = nn.Conv2d(in_channels, c2_in, kernel_size=1)
        self.b2_2 = nn.Conv2d(c2_in, c2_out, kernel_size=3, padding=1)

        # Branch 3
        self.b3_1 = nn.Conv2d(in_channels, c3_in, kernel_size=1)
        self.b3_2 = nn.Conv2d(c3_in, c3_out, kernel_size=5, padding=2)

        # Branch 4
        self.b4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.b4_2 = nn.Conv2d(in_channels, c4_out, kernel_size=1)

    def forward(self, x):
        b1 = F.relu(self.b1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(self.b4_1(x)))
        return torch.cat((b1, b2, b3, b4), dim=1)

#Model GoogleNet

In [37]:
# GoogleNet model implementation
class GoogleNet(nn.Module):
    def __init__(self, num_classes=10):
        super(GoogleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3)  # Stem
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.conv2_1 = nn.Conv2d(64, 64, kernel_size=1)  # Inception 3a
        self.conv2_2 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.maxpool1(F.relu(self.conv1(x)))
        x = self.maxpool2(F.relu(self.conv2_2(F.relu(self.conv2_1(x)))))
        x = self.maxpool3(self.inception3b(self.inception3a(x)))
        x = self.maxpool4(self.inception4e(self.inception4d(self.inception4c(self.inception4b(self.inception4a(x))))))
        x = self.avgpool(self.inception5b(self.inception5a(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

#Cek Output

In [38]:
# Mengecek bentuk output dari setiap inception block
def check_output_shape(model, input_size):
    model.eval()
    with torch.no_grad():
        x = torch.randn(*input_size)
        output = model(x)
        print(f"Output shape: {output.shape}")

#Hasil

In [39]:
# Membuat model GoogleNet
model = GoogleNet(num_classes=10)

# Menampilkan bentuk output dari setiap inception block
check_output_shape(model, (1, 1, 96, 96))

Output shape: torch.Size([1, 10])
